# Parallel Optimization with APOSMM

This tutorial demonstrates libEnsemble’s capability to identify multiple minima from simulation outputs using the built-in APOSMM (Asynchronously Parallel Optimization Solver for finding Multiple Minima) generator function (`gen_f`). In this tutorial, we’ll create a simple simulation function (`sim_f`) that defines a function with multiple minima, then write a libEnsemble calling script that imports APOSMM and parameterizes it to check for minima over a domain of outputs from our `sim_f`.

Besides libEnsemble and NumPy, SciPy and mpmath are also required dependencies.

**Note that for notebooks** the multiprocessing start method should be set to `fork` (default on Linux). To use
with `spawn` (default on Windows and macOS), use the `multiprocess` library.

Let's make sure libEnsemble is installed.

In [ ]:
!pip install libensemble

## Six-Hump Camel Simulation Function

Describing APOSMM’s operations is simpler with a given function on which to depict evaluations. We’ll use the Six-Hump Camel function, known to have six global minima. A sample space of this function, containing all minima, appears below:

![6humpcamel](https://raw.githubusercontent.com/Libensemble/libensemble/main/docs/images/basic_6hc.png)


In [ ]:
# Define our simulation function
import numpy as np


def six_hump_camel(H, persis_info, sim_specs, _):
    """Six-Hump Camel sim_f."""

    batch = len(H["x"])  # Num evaluations each sim_f call.
    H_o = np.zeros(batch, dtype=sim_specs["out"])  # Define output array H

    for i, x in enumerate(H["x"]):
        H_o["f"][i] = six_hump_camel_func(x)  # Function evaluations placed into H

    return H_o, persis_info


def six_hump_camel_func(x):
    """Six-Hump Camel function definition"""
    x1 = x[0]
    x2 = x[1]
    term1 = (4 - 2.1 * x1**2 + (x1**4) / 3) * x1**2
    term2 = x1 * x2
    term3 = (-4 + 4 * x2**2) * x2**2

    return term1 + term2 + term3

## APOSMM Operations

APOSMM coordinates multiple local optimization runs starting from a collection of sample points. These local optimization runs occur concurrently, and can incorporate a variety of optimization methods, including from NLopt, PETSc/TAO, and SciPy. Some number of uniformly sampled points is returned by APOSMM for simulation evaluations before local optimization runs can occur, if no prior simulation evaluations are provided. User-requested sample points can also be provided to APOSMM:

![6hcsampling](https://raw.githubusercontent.com/Libensemble/libensemble/main/docs/images/sampling_6hc.png)

Specifically, APOSMM will begin local optimization runs from those points that don’t have better (more minimal) points nearby within a threshold ``r_k``. For the above example, after APOSMM has returned the uniformly sampled points, for simulation evaluations it will likely begin local optimization runs from the user-requested approximate minima. Providing these isn’t required, but can offer performance benefits.

Each local optimization run chooses new points and determines if they’re better by passing them back to be evaluated by the simulation routine. If so, new local optimization runs are started from those points. This continues until each run converges to a minimum:

![6hclocalopt](https://raw.githubusercontent.com/Libensemble/libensemble/main/docs/images/localopt_6hc.png)

Throughout, generated and evaluated points are appended to the History array, with the field ``'local_pt'`` being ``True`` if the point is part of a local optimization run, and ``'local_min'`` being ``True`` if the point has been ruled a local minimum.

## APOSMM Persistence

APOSMM uses a Persistent generator. A single worker process initiates APOSMM so that it “persists” and keeps running over the course of the entire libEnsemble routine.

APOSMM begins its own concurrent optimization runs, each of which independently produces a sequence of points trying to find a local minimum. These points are given to workers and evaluated by simulation routines.

If there are more workers than optimization runs at any iteration of the generator, additional random sample points are generated to keep the workers busy.


## Calling Script

Start by importing NumPy, libEnsemble routines, APOSMM, our ``sim_f``, and a specialized allocation function:

In [ ]:
import numpy as np

# from tutorial_six_hump_camel import six_hump_camel

from libensemble.libE import libE
from libensemble.gen_funcs.persistent_aposmm import aposmm
from libensemble.alloc_funcs.persistent_aposmm_alloc import persistent_aposmm_alloc
from libensemble.tools import parse_args, add_unique_random_streams

This allocation function starts a single Persistent APOSMM generator to generate points (simulation input parameters), and returns the resulting values from each simulation (run in parallel). Points can be sampled points or points from the parallel local optimization runs.

APOSMM supports a wide variety of external optimizers. The following statement sets the optimizer settings to ``'scipy'`` to indicate to APOSMM which optimization method to use, so it is imported at global scope:

In [ ]:
import libensemble.gen_funcs

libensemble.gen_funcs.rc.aposmm_optimizers = "scipy"

This script uses the dictionary interface to configure the run (the newer object interface is equally valid).
Set up ``nworkers``, ``libE_specs``, ``sim_specs``, ``gen_specs``, and ``alloc_specs``:

In [ ]:
nworkers = 4  # One for persistent generator and three for simulations
libE_specs = {"nworkers": nworkers, "comms": "local"}

sim_specs = {
    "sim_f": six_hump_camel,  # Simulation function
    "in": ["x"],  # Accepts 'x' values
    "out": [("f", float)],
}  # Returns f(x) values

gen_out = [
    ("x", float, 2),  # Produces 'x' values
    ("x_on_cube", float, 2),  # 'x' values scaled to unit cube
    ("sim_id", int),  # Produces sim_id's for History array indexing
    ("local_min", bool),  # Is a point a local minimum?
    ("local_pt", bool),  # Is a point from a local opt run?
]

gen_specs = {
    "gen_f": aposmm,  # APOSMM generator function
    "persis_in": ["f", "x", "x_on_cube", "sim_id", "local_min", "local_pt"],  # Fields we want to pass back to APOSMM
    "out": gen_out,  # Output defined like above dict
    "user": {
        "initial_sample_size": 100,  # Random sample 100 points to start
        "localopt_method": "scipy_Nelder-Mead",
        "opt_return_codes": [0],  # Return code specific to localopt_method
        "max_active_runs": 6,  # Maximum concurrent local optimization runs
        "lb": np.array([-2, -1]),  # Lower bound of search domain
        "ub": np.array([2, 1]),  # Upper bound of search domain
    },
}

alloc_specs = {"alloc_f": persistent_aposmm_alloc}

exit_criteria = {"sim_max": 800}

``gen_specs['user']`` fields above that are required for APOSMM are:

* ``'lb'`` - Search domain lower bound
* ``'ub'`` - Search domain upper bound
* ``'localopt_method'`` - Chosen local optimization method
* ``'initial_sample_size'`` - Number of uniformly sampled points generated
  before local optimization runs.
* ``'opt_return_codes'`` - A list of integers that local optimization
  methods return when a minimum is detected. SciPy's Nelder-Mead returns 0,
  but other methods (not used in this tutorial) return 1.

Also note the following:

* ``'x_on_cube'`` in ``gen_specs['out']``. APOSMM works internally on
  ``'x'`` values scaled to the unit cube. To avoid back-and-forth scaling
  issues, both types of ``'x'``'s are communicated back, even though the
  simulation will likely use ``'x'`` values. (APOSMM performs handshake to
  ensure that the ``x_on_cube`` that was given to be evaluated is the same
  one that is given back.)
* ``'sim_id'`` in ``gen_specs['out']``. APOSMM produces points in it's
  local History array that it will need to update later, and can best
  reference those points (and avoid a search) if APOSMM produces the IDs
  itself, instead of libEnsemble.

Other options and configurations can be found in the APOSMM [API reference](https://libensemble.readthedocs.io/en/main/examples/aposmm.html).

The ``exit_criteria`` tells libEnsemble when to stop.

## Run the Ensemble

Finally, set persis_info (to provide random seeds to workers) and run the ensemble.

In [ ]:
persis_info = add_unique_random_streams({}, nworkers + 1)

H, persis_info, flag = libE(sim_specs, gen_specs, exit_criteria, persis_info, alloc_specs, libE_specs)

In [ ]:
print("Minima:", H[np.where(H["local_min"])]["x"])

## Output

Please note that one worker will be “persistent” for APOSMM for the duration of the routine.

The output from the above print should resemble:

    Minima:[[ 0.08978611 -0.71258202]
    [-1.70380373  0.79614422]
    [-0.08985348  0.71271365]
    [ 1.70352649 -0.79610854]
    [ 1.60712922  0.56868726]
    [-1.60711706 -0.56868708]
    [ 1.70361564 -0.7960268 ]]
     
The first six values correspond to the local minima for the Six-Hump Camel simulation function.

The 7th value is a repeat minimum, as APOSMM will continue to start local optimization runs.

## Viewing Animation

The following cell produces a 3D animation showing the random sampling points, 
the points produced by the optimization runs, and the local Minima.

This may take up to about 30 seconds to produce the 3D animation, depending on system.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from matplotlib.lines import Line2D

def animate_aposmm_3d(H, batch_size):
    x_vals = np.linspace(-2, 2, 50)
    y_vals = np.linspace(-1, 1.1, 50)
    X, Y = np.meshgrid(x_vals, y_vals)
    Z = np.array([six_hump_camel_func(np.array([x, y])) for x, y in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection="3d")
    ax.plot_surface(X, Y, Z, cmap="winter", edgecolor='k', linewidth=0.1, antialiased=True, alpha=0.5) 
    sc_normal = ax.scatter3D([], [], [], s=6, color="black", marker="o", label="Point")
    sc_localp = ax.scatter3D([], [], [], s=40, color="red", marker="^", label="Optimization point")
    custom_M_marker = Line2D([0], [0], linestyle='None', marker='$\\mathrm{M}$',
                             markersize=8, markerfacecolor='black', markeredgecolor='black', color='white')
    ax.legend([sc_normal, sc_localp, custom_M_marker], ["Point", "Optimization point", "Local minimum"],loc="upper left")
    fig.tight_layout()
    annotations = []

    def update(frame):
        for ann in annotations:
            ann.remove()
        annotations.clear()
        end = min((frame + 1) * batch_size, len(H))
        H_sub = H[:end]
        masks = [~H_sub["local_pt"] & ~H_sub["local_min"], H_sub["local_pt"], H_sub["local_min"]]
        (x_n, y_n, f_n), (x_lp, y_lp, f_lp), (x_lm, y_lm, f_lm) = [
            (H_sub["x"][m, 0], H_sub["x"][m, 1], H_sub["f"][m]) for m in masks
        ]
        sc_normal._offsets3d = (x_n, y_n, f_n)
        sc_localp._offsets3d = (x_lp, y_lp, f_lp)
        for i in range(len(x_lm)):
            annotations.append(ax.text(x_lm[i], y_lm[i], f_lm[i], "M", color="white", fontsize=12,
                bbox=dict(facecolor="black", alpha=0.7, pad=2), zorder=999
            ))
        return sc_normal, sc_localp
    total_frames = (len(H) + batch_size - 1) // batch_size
    ani = animation.FuncAnimation(fig, update, frames=total_frames, interval=500, blit=False, repeat=False)
    plt.close(fig)
    return HTML(ani.to_jshtml())

# Reduce batch_size for more refined steps
animate_aposmm_3d(H, batch_size=50)

## Applications

APOSMM is not limited to evaluating minima from pure Python simulation functions.
Many common libEnsemble use-cases involve using libEnsemble's Executor to launch user
applications with parameters requested by APOSMM, then evaluate their output using
APOSMM, and repeat until minima are identified. A currently supported example
can be found in libEnsemble's [WarpX Scaling Test](https://github.com/Libensemble/libe-community-examples/tree/main/warpx)

Please see the [API reference](https://libensemble.readthedocs.io/en/main/examples/aposmm.html) for more APOSMM configuration options and other information.